In [ ]:
import json
import csv

In [ ]:
cpsat_sol = []
with open('cpsat_solutions.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        cpsat_sol.append(row)

In [ ]:
def compare_results(cpsat_infer):
    comparison_results = []
    for test_bench_id, test_result in cpsat_infer.items():
        row = 0
        while int(cpsat_sol[row][0]) != test_result['benchmark_id']:
            row += 1
        bench_row_start = row
        while int(cpsat_sol[row][3]) > test_result['feasibility_abs_makespan'] and \
            int(cpsat_sol[row][0]) == test_result['benchmark_id']:
            row += 1
        if row > bench_row_start and int(cpsat_sol[row][0]) == test_result['benchmark_id']:
            cpsat_same_quality_time = 0.5 * (float(cpsat_sol[row - 1][1]) + float(cpsat_sol[row][1]))
        else:
            cpsat_same_quality_time = float(cpsat_sol[row][1])
        infer_feasible_time = test_result['inference_time'] + test_result['feasibility_timing']
        comparison_results.append({
            'benchmark_id': test_result['benchmark_id'],
            'infer_feasible_time': infer_feasible_time,
            'cpsat_same_quality_time': cpsat_same_quality_time,
            'rel_time_savings_per': infer_feasible_time / cpsat_same_quality_time * 100.0
        })
    return comparison_results

### Unconstrained makespan case

In [ ]:
with open('inference-sgs_vs_cpsat.json', 'r') as jsonfile:
    cpsat_infer = json.load(jsonfile)
    comparison_results = compare_results(cpsat_infer)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib widget

rel_time_savings_gain = [res['rel_time_savings_per'] for res in comparison_results if res['rel_time_savings_per'] < 100.]
timings = [(res['infer_feasible_time'], res['cpsat_same_quality_time']) for res in comparison_results if res['rel_time_savings_per'] < 100.]
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize =(10, 7))
ax1.set_xlabel('Test RCPSP benchmarks')
ax1.set_ylabel('Relative computation time savings for same quality (%)')
ax1.scatter(x=np.arange(len(rel_time_savings_gain)), y=rel_time_savings_gain)
ax2.set_xlabel('Test RCPSP benchmarks')
ax2.set_ylabel('Computation time to reach same quality (s.)')
ax2.plot(np.arange(len(rel_time_savings_gain)), [t[0] for t in timings], label='GNN + SGS')
ax2.plot(np.arange(len(rel_time_savings_gain)), [t[1] for t in timings], label='CP-optimize')
ax2.legend()
plt.show()

In [ ]:
rel_time_savings_loss = [res['rel_time_savings_per'] for res in comparison_results if res['rel_time_savings_per'] >= 100.]
timings = [(res['infer_feasible_time'], res['cpsat_same_quality_time']) for res in comparison_results if res['rel_time_savings_per'] >= 100.]
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize =(10, 7))
ax1.set_xlabel('Test RCPSP benchmarks')
ax1.set_ylabel('Relative computation time losses for same quality (%)')
ax1.scatter(x=np.arange(len(rel_time_savings_loss)), y=rel_time_savings_loss)
ax2.set_xlabel('Test RCPSP benchmarks')
ax2.set_ylabel('Computation time to reach same quality (s.)')
ax2.plot(np.arange(len(rel_time_savings_loss)), [t[0] for t in timings], label='GNN + SGS')
ax2.plot(np.arange(len(rel_time_savings_loss)), [t[1] for t in timings], label='CP-optimize')
ax2.legend()
plt.show()

In [ ]:
len(rel_time_savings_gain) / len(comparison_results) * 100.

In [ ]:
len(rel_time_savings_loss) / len(comparison_results) * 100.

In [ ]:
fig, ax = plt.subplots(figsize =(10, 7))
ax.hist(rel_time_savings_gain + rel_time_savings_loss, bins=np.arange(start=0, stop=1000, step=10))
plt.show()